# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition.txt". This file is coming from the _Validation/Geometry_ package and more details on the setup and the procedure can be found here: 

https://twiki.cern.ch/twiki/bin/view/Sandbox/HGCalMaterialBudget

In [1]:
inputfile = "VolumesZPosition.txt"

## Some necessary imports

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display

## Materials dEdx and radiation lengths

In [3]:
#In MeV/mm
dEdx = OrderedDict()
#--------
#Some elements necessary to build our materials
dEdx['Fe'] = 1.143
dEdx['Mn'] = 1.062
dEdx['Cr'] = 1.046
dEdx['Ni'] = 1.307
dEdx['C']  = 0.3952
dEdx['0']  = 0. # 2.398E-04 -> essentially zero
dEdx['H']  = 0. #3.437E-05 -> essentially zero
dEdx['Br'] = 0. #9.814E-04 -> essentially zero
dEdx['W']  = 2.210
#-------- 

dEdx['Copper'] = 1.257
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#1996
dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
dEdx['Silicon'] = 0.3876
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2730
dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
dEdx['Other'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0290
dEdx['Air'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#3692
dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0568
dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
#--------
dEdx['Lead'] = 1.274 #Pb
#Composition of cable as Sunanda uses them is here: 
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2841
dEdx['Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']

#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['Cables'] = 66.722
MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118

In [4]:
# Let's read the input file
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R"])

In [5]:
# Drop duplicates and sort values starting from far left to far right
#matZ = matZ.drop_duplicates(subset=['Mat', 'Z'], keep='first')#.sort_values('Z', ascending=[True])
#matZ = matZ.sort_values('Z', ascending=[True])

In [6]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(-1)

In [7]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [8]:
# Now that we have the tracknum we will not let the last Copper volume to 
# destroy our calculation with its huge thickness due to track change. 
#According to TDR BH back is at 5137.7 mm so we put 6.0 mm for that Copper
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 6.0

In [9]:
# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [10]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [11]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [12]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [13]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [14]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that right before the next layer "Mat" column will be either 
# Silicon or Scintillator. 
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = (( matZ["layerflag"] == False) & (matZ["layerflag"].shift() == True))
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)
#matZ = matZ.drop('layerflag', 1)

In [15]:
# Drop auxillary columns
matZ = matZ.drop('layerflag', 1)

In [16]:
# There are some tracks that have 53 layers due to the fact 
# that they pass in the mixed layers from both silicon and scintillator.
# Let's filter them out. Be careful! Filter chooses and not disgards!
matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [17]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
#matZ[matZ["tracknum"] == 6.0 ]
matZ
#matZ[matZ["layer"] == 52]

,Mat,Z,Eta,R,PhysThickInmm,tracknum,PhysThickInXo,dEdx,dEdxtimesdx,dEdxtimesdxCum,PhysThickInXoCum,layer
988,StainlessSteel,3190.5,2.14618,756.472,0.3,2.0,1.728559e-02,1.139861,0.341958,0.341958,0.017286,1.0
989,Lead,3190.8,2.14618,756.543,2.1,2.0,3.742115e-01,1.274000,2.675400,3.017358,0.391497,1.0
990,Copper,3192.9,2.14618,757.041,0.1,2.0,6.965777e-03,1.257000,0.125700,3.143058,0.398463,1.0
991,StainlessSteel,3193.0,2.14618,757.065,0.3,2.0,1.728559e-02,1.139861,0.341958,3.485017,0.415748,1.0
992,M_NEMA_FR4_plate,3193.3,2.14618,757.136,1.6,2.0,9.139932e-03,0.179950,0.287920,3.772937,0.424888,1.0
993,Air,3194.9,2.14618,757.516,1.5,2.0,4.974761e-06,0.000000,0.000000,3.772937,0.424893,1.0
994,M_NEMA_FR4_plate,3196.4,2.14618,757.871,1.6,2.0,9.139932e-03,0.179950,0.287920,4.060857,0.434033,1.0
995,Air,3198.0,2.14618,758.251,0.3,2.0,9.949523e-07,0.000000,0.000000,4.060857,0.434034,1.0
996,WCu,3198.3,2.14618,758.322,1.4,2.0,2.733041e-01,1.971750,2.760450,6.821307,0.707338,1.0
997,Copper,3199.7,2.14618,758.654,6.0,2.0,4.179466e-01,1.257000,7.542000,14.363307,1.125285,1.0


In [18]:
#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]

In [19]:
# We will add a column with the dedx weights
# First dedxtimesdx sum per layer but not including silicon or scintillator
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 2.0  ]

,tracknum,layer,dEdxtimesdx,dedxweights
0,2.0,1.0,17.123757,12.680875
1,2.0,2.0,8.237994,10.708587
2,2.0,3.0,13.179180,10.708587
3,2.0,4.0,8.237994,10.708587
4,2.0,5.0,13.179180,10.708587
5,2.0,6.0,8.237994,10.708587
6,2.0,7.0,13.179180,10.708587
7,2.0,8.0,8.237994,10.708587
8,2.0,9.0,13.179180,10.708587
9,2.0,10.0,8.237994,10.708587
